# Input data and temperature sensitivity

A continuation of the simple sensitivity study we did in 
[Sensitivity studies using COSIPY](sensitivity_study.ipynb) is to investigate how changing the surface temperature, or any other input variable, affects the calculation of the surface mass balance. COSIPY reads the meteorological input data from a **netcdf** file during the run. This means that during each time step the model reads the meteorological data from the corresponding time step in the input file. The input file can be either "1D" for a point simulation or "2D" for a distributed simulation. The input file can be based on either observed or modeled data.

In order to conduct the temperature sensitivity study (or any study) from scratch we need to do the following:
- Create the input file from observations.
- Load the input file into a xarray dataset.
- Create copies of this dataset and add the bias to the variable in question.

With this done we can run the simulations. But before we do any of this, we can take a look at the input data for the Zhadang glacier, which is shipped with COSPIY for theses tutorials.

**The standard imports**

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
import sys
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')
sys.path.append(os.getcwd())

In [ ]:
from cosipy.utils import edu_utils
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

The path to the input netcdf is stored in the `input_netcdf` variable. We can take a look at the options for the current value.

In [ ]:
# print options returns a pandas dataframe with the variable as the index,
# hence we can index it.
edu_utils.print_options().loc['input_netcdf']

We can then open the file with xarray:

In [ ]:
# Files are located in ./data/ relative to the root directory of cosipy. 
# This is also our current working directory.
input_path = './data/' + 'input/' +\
            edu_utils.OPTIONS['input_netcdf']
with xr.open_dataset(input_path) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

As you can see, this file contain variables such as the 2-meter temperature (T2), the relative humidity at 2 meters (RH2) and the cloud cover (N) for one single point. The time coordinate spans 2009-01-01 00:00:00 and 2009-01-31 22:00:00 at a hourly resolution.


<div class="alert alert-warning">
<details>
    <summary><b>Question: Can you figure out what the variable G is and what unit it has?</b> <i>Click me for a hint!</i></summary>
    Try pressing the document symbol to the right in the output above, this shows some extra information about the variable.
    </details>
</div>

We can quickly take a look at one of the variables by plotting it

In [ ]:
ds.T2.plot(figsize=(10,5));

## Creating the input files

In this case COSIPY comes packaged with the processed data which can directly be used to drive a simulation. In a more realistic scenario however, you probably want drive COSIPY with your own data from another glacier than the Zhadang glacier. The `aws2cosipy` module is provided by COSIPY to aid the processing of .csv-files from weather stations or nwp-output into input files which can be used by COSIPY. 

In [ ]:
# We import the create_1D_input function to process a single point
from utilities.aws2cosipy.aws2cosipy import create_1D_input

The `create_1D_input` has five arguments: the csv file to process, the name of the resulting input file, the name of a static file describing the altitude, slope and aspect of the grid point, and the start and end date.

In our case the .csv file is located in the same directory as the input file we looked at earlier. It is called `Zhadang_ERA5_2009_2018.csv`. 

In [ ]:
# Path to the file
file = './data/input/Zhadang/Zhadang_ERA5_2009_2018.csv'

In [ ]:
# Static file
static_file = './data/static/Zhadang_static.nc'

In [ ]:
# Define time start and end.
start_date = '20120701'
end_date = '20120731'

In [ ]:
# Output file, we change this for our year.
output_name = './data/input/Zhadang/Zhadang_ERA5_2012_test.nc'

In [ ]:
# Call the function, this takes some time.
create_1D_input(file, output_name, static_file, start_date, end_date)

Open the file to see that it worked

In [ ]:
with xr.open_dataset(output_name) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

This created a input file with data for the month of July in 2012. The .csv-file contains data from January 2000 until early January 2018, so you are free to change the start and end data accordingly.

## Creating datasets for temperature bias experiments

A temperature bias experiment is essentially a sensitivity study. It explores the effects of changing the temperature by $n$ degrees over the whole time period. For example, let's say we want to know how the glacier responds to a climate which is  2 °C warmer. This approach isolates the effect of temperature on the glacier and does not take other factors, like a changing hydrological cycle, into account.

Setting this up with COSIPY follows a similar approach as described in the [Sensitivity studies with COSIPY](sensitivity_study.ipynb). However, instead of manipulating the the constants of the model with the `opt_dict` we are changing the dataset containing the input data before sending it to the `cosipy_core`.

If we want to use our newly processed input file, we have to change the `input_netcdf` in the `opt_dict` along with the integration time (`time_start` & `time_end`) since this have to match the data. After this we can initiate the IOClass and datasets

In [ ]:
opt_dict = {'input_netcdf': 'Zhadang/Zhadang_ERA5_2012_test.nc',
            'time_start': '2012-07-01T00:00',
            'time_end': '2012-07-31T00:00'}

In [ ]:
# Default
IO_def, DATA_def, RESULTS_def = edu_utils.create_IO(opt_dict)
# Up
IO_up, DATA_up, RESULTS_up = edu_utils.create_IO(opt_dict)
# Down
IO_dn, DATA_dn, RESULTS_dn = edu_utils.create_IO(opt_dict)

And then apply the bias to the input data

In [ ]:
# Bias is two degrees
bias = 2
DATA_dn['T2'] = DATA_dn['T2'] - bias
DATA_up['T2'] = DATA_up['T2'] + bias

**Plot it to make sure it worked**

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
DATA_def['T2'].plot(ax=ax, label='Default')
DATA_up['T2'].plot(ax=ax, label='+2')
DATA_dn['T2'].plot(ax=ax, label='-2')
plt.legend();

### Setting up the run
As in the previous sensitivity study we put everything in a list of lists. Note that we're not  any of the options this time, only the input data and results dataset.

In [ ]:
# List of lists with our experiments
exp_list = [[DATA_def, IO_def, RESULTS_def],
            [DATA_dn, IO_dn, RESULTS_dn],
            [DATA_up, IO_up, RESULTS_up]
           ]

And lets runs the experiments

In [ ]:
for exp in exp_list:
    # Call run_model once for each experiment
    edu_utils.run_model(DATA=exp[0], IO=exp[1], RESULT=exp[2])

We select a few days in the middle of the month to plot

In [ ]:
labels = ['Default', '-2$\degree$C', '+2$\degree$C']
fig, ax = plt.subplots(figsize=(10, 5))
for exp, label in zip(exp_list, labels):
    # Get the data and plot it RESULTS are kept at the third spot, index 2
    exp[2].sel(time=slice('2012-07-07', '2012-07-12')).surfMB.plot(ax=ax,
                                                                   label=label)
plt.legend(); 

<div class="alert alert-warning">
    <details>
        <summary>
            <b>Question: What is the difference between the experiments in the total mass lost during the period plotted above?</b> <i>Click me for an explanation</i>
        </summary>
        Try selecting the data from the experiment list and then calculate the sum with .sum(). Alternatively you can also copy the cell above and add .cumsum() before .plot(). This will plot the cumulative sum over the period.
    </details>
</div>

In [ ]:
# Here you can write some code. Click to open.


## Next steps
[Back to overview](welcome.ipynb)

[Distributed simulations](distributed_run.ipynb)
